In [3]:
import cv2
import numpy as np
import scipy
from scipy.misc import imread
import pickle
import random
import os
import matplotlib.pyplot as plt

In [5]:
image_path = 'queryImages/sample_1.jpg'
database_path = 'referenceImages/'

In [21]:
import glob
import cv2

# Feature extractor
def extract_features(image_path, vector_size=32):
    image = imread(image_path, mode="RGB")
    try:
        alg = cv2.KAZE_create()
        # Finding image keypoints
        # use glob to grab the image paths and loop over them
        imagePaths = [f for f in glob.glob('directory/*.png')]
        kps = alg.detect(image)
        # Getting first 32 of them. 
        # Number of keypoints is varies depend on image size and color pallet
        # Sorting them based on keypoint response value(bigger is better)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        # computing descriptors vector
        kps, dsc = alg.compute(image, kps)
        # Flatten all of them in one big vector - our feature vector
        dsc = dsc.flatten()
        # Making descriptor of same size
        # Descriptor vector size is 64
        needed_size = (vector_size * 64)
        if dsc.size < needed_size:
            # if we have less the 32 descriptors then just adding zeros at the
            # end of our feature vector
             dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
    except cv2.error as e:
        print ('Error: ', e)
        return None

    return dsc


def batch_extractor(database_path, pickled_db_path="kaze_features.pck"):
    files = [os.path.join(database_path, p) for p in sorted(os.listdir(database_path))]

    result = {}
    for f in files:
        print ('Extracting features from image %s' % f)
        name = f.split('/')[-1].lower()
        result[name] = extract_features(f)
    
    # saving all our feature vectors in pickled file
    with open(pickled_db_path, 'wb') as fp:
#         pickle.dump(result, fp)

In [22]:
class Matcher(object):

    def __init__(self, pickled_db_path="kaze_features.pck"):
        with open(pickled_db_path) as fp:
#             self.data = pickle.load(fp)
        self.names = []
        self.matrix = []
        for k, v in self.data.iteritems():
            self.names.append(k)
            self.matrix.append(v)
        self.matrix = np.array(self.matrix)
        self.names = np.array(self.names)

    def cos_cdist(self, vector):
        # getting cosine distance between search image and images database
        v = vector.reshape(1, -1)
        return scipy.spatial.distance.cdist(self.matrix, v, 'cosine').reshape(-1)

    def match(self, image_path, topn=5):
        features = extract_features(image_path)
        img_distances = self.cos_cdist(features)
        # getting top 5 records
        nearest_ids = np.argsort(img_distances)[:topn].tolist()
        nearest_img_paths = self.names[nearest_ids].tolist()

        return nearest_img_paths, img_distances[nearest_ids].tolist()

In [24]:

def show_img(path):
    img = imread(path, mode="RGB")
    plt.imshow(img)
    plt.show()
    
def run():
    images_path = database_path
    files = [os.path.join(images_path, p) for p in sorted(os.listdir(images_path))]
    # getting 3 random images 
    sample = random.sample(files, 3)
    
    batch_extractor(images_path)

    ma = Matcher('kaze_features.pck')
    
    for s in sample:
        print ('Query image ==========================================')
        show_img(s)
        names, match = ma.match(s, topn=3)
        print ('Result images ========================================')
        for i in range(3):
            # we got cosine distance, less cosine distance between vectors
            # more they similar, thus we subtruct it from 1 to get match value
            print ('Match %s' % (1-match[i]))
            show_img(os.path.join(images_path, names[i]))

run()

Extracting features from image referenceImages/1792-bottle-in-bond-[size].png


/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


Extracting features from image referenceImages/1792-full-proof-bourbon-[size].png
Extracting features from image referenceImages/1792-high-rye-[size].png
Extracting features from image referenceImages/1792-single-barrel-bourbon-[size].png
Extracting features from image referenceImages/1792-small-batch-bourbon-[size].png
Extracting features from image referenceImages/1800-anejo-[size].png
Extracting features from image referenceImages/1800-reposado-[size].png
Extracting features from image referenceImages/1800-silver-[size].png
Extracting features from image referenceImages/1921-anejo-[size].png
Extracting features from image referenceImages/1921-reposado-[size].png
Extracting features from image referenceImages/Ad-Laws-bottled-in-bond.png
Extracting features from image referenceImages/Ad-Laws-straight-bourbon-whiskey.png
Extracting features from image referenceImages/Angel's Kentucky Straight Bourbon.png
Extracting features from image referenceImages/Angels-Envy-Cask-Strength.png
Extra

Extracting features from image referenceImages/MYERSS_RUM_ORIGINAL_DARK_RUM_80PF_750ML.png
Extracting features from image referenceImages/Mezcales de Leyenda - durango.png
Extracting features from image referenceImages/Mezcales de Leyenda - guerrero.png
Extracting features from image referenceImages/Mezcales de Leyenda - oaxaca.png
Extracting features from image referenceImages/Milagro Anejo Select Barrel.png
Extracting features from image referenceImages/Milagro Reposado.png
Extracting features from image referenceImages/Milagro Silver.png
Extracting features from image referenceImages/Mount Gay XO Reserve Cask.png
Extracting features from image referenceImages/NIKKA-COFFEY-GRAIN-MALT-70CL.png
Extracting features from image referenceImages/Nikka-Coffey-Malt.png
Extracting features from image referenceImages/Nikka-from-the-barrel-whisky-500ml.png
Extracting features from image referenceImages/Oban14-750ml.png
Extracting features from image referenceImages/Oban_Distiller_s_Edition_750ml

Extracting features from image referenceImages/armorik-single-malt-[size].png
Extracting features from image referenceImages/armorik-single-malt-sherry-finish-[size].png
Extracting features from image referenceImages/auchentoshan-12-year-[size].png
Extracting features from image referenceImages/auchentoshan-18-year-[size].png
Extracting features from image referenceImages/auchentoshan-21-year-[size].png
Extracting features from image referenceImages/auchentoshan-american-oak-[size].png
Extracting features from image referenceImages/auchentoshan-three-woods-[size].png
Extracting features from image referenceImages/auchentoshan-virgin-oak-[size].png
Extracting features from image referenceImages/aultmore-12-year-[size].png
Extracting features from image referenceImages/aultmore-18-year-[size].png
Extracting features from image referenceImages/averna-amaro-[size].png
Extracting features from image referenceImages/aviation-american-gin-[size].png
Extracting features from image referenceIma

...

Extracting features from image referenceImages/glen-grant-five-decades-[size].png
Extracting features from image referenceImages/glen-moray-12-year-[size].png
Extracting features from image referenceImages/glen-moray-elgin-classic-port-cask-finish-[size].png
Extracting features from image referenceImages/glendronach-15-year-single-malt-scotch-whiskey-[size].png
Extracting features from image referenceImages/glendronach-18-year-single-malt-scotch-whiskey-[size].png
Extracting features from image referenceImages/glendronach-19-year-1995-2014-oloroso-sherry-cask-[size].png
Extracting features from image referenceImages/glendronach-21-year-single-malt-scotch-whiskey-[size].png
Extracting features from image referenceImages/glendronach-27-year-1990-2017-sherry-butt-cask-[size].png
Extracting features from image referenceImages/glendullan-singleton-18-year-[size].png
Extracting features from image referenceImages/glenfiddich-12-year-[size].png
Extracting features from image referenceImages/g

Extracting features from image referenceImages/joseph-magnus-bourbon-[size].png
Extracting features from image referenceImages/kaiyo-mizunara-oak-cask-strength-[size].png
Extracting features from image referenceImages/kaiyo-mizunara-oak-the-peated-[size].png
Extracting features from image referenceImages/kaiyo-mizunara-oak-the-single-[size].png
Extracting features from image referenceImages/kaiyo-mizunara-oak-unchillfiltered-[size].png
Extracting features from image referenceImages/kappa-pisco-[size].png
Extracting features from image referenceImages/kavalan-amontillado-[size].png
Extracting features from image referenceImages/kavalan-classic-[size].png
Extracting features from image referenceImages/kavalan-manzanilla-solist-[size].png
Extracting features from image referenceImages/kavalan-sherry-oak-[size].png
Extracting features from image referenceImages/kavalan-solist-moscatel-solist-[size].png
Extracting features from image referenceImages/kelt-vsop-[size].png
Extracting features 

Extracting features from image referenceImages/old-forester-1920-prohibition-style-[size].psd
Extracting features from image referenceImages/old-grand-dad-114-[size].png
Extracting features from image referenceImages/old-grand-dad-bourbon-100-proof.png
Extracting features from image referenceImages/old-grand-dad-bourbon-[size].png
Extracting features from image referenceImages/old-maysville-club-rye-[size].png
Extracting features from image referenceImages/old-medley-12-year-[size].png
Extracting features from image referenceImages/old-overholt-rye-[size].png
Extracting features from image referenceImages/old-pogue-masters-select-[size].png
Extracting features from image referenceImages/old-pulteney-35-year-[size].png
Extracting features from image referenceImages/omar-single-malt-bourbon-cask-[size].png
Extracting features from image referenceImages/omar-single-malt-sherry-cask-[size].png
Extracting features from image referenceImages/osocalis-apple-brandy-barrel-aged-[size].png
Extra

Extracting features from image referenceImages/teachers-highland-cream-blended-scotch-whiskey-[size].png
Extracting features from image referenceImages/tears-of-llorona-extra-anejo-M.png
Extracting features from image referenceImages/teeling-single-grain-[size].png
Extracting features from image referenceImages/teeling-small-batch-irish-whiskey-[size].png
Extracting features from image referenceImages/templeton-rye-whiskey-[size].png
Extracting features from image referenceImages/tequila-ocho-plata-[size].png
Extracting features from image referenceImages/tequila-ocho-reposado-[size].png
Extracting features from image referenceImages/tesseron-lot-53-xo-cognac-[size].png
Extracting features from image referenceImages/tesseron-lot-65-xo-cognac-[size].png
Extracting features from image referenceImages/tesseron-lot-76-xo-cognac-[size].png
Extracting features from image referenceImages/tesseron-lot-90-xo-cognac-[size].png
Extracting features from image referenceImages/the-glenlivet-12-year-

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte